### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
nyt_api_query = url + "fq=" + filter_query \
+ "&sort=" + sort \
+ "&fl=" + field_list \
+ "&api-key=" + nyt_api_key \
+ "&begin_date=" + begin_date \
+ "&end_date=" + end_date

print(nyt_api_query)

https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&api-key=pyj2nUbg2J4HL7RMXNqNCjBm4xySsvlN&begin_date=20130101&end_date=20230531


In [4]:
test = requests.get(nyt_api_query).json()
print(json.dumps(test, indent=4))

{
    "status": "OK",
    "copyright": "Copyright (c) 2024 The New York Times Company. All Rights Reserved.",
    "response": {
        "docs": [
            {
                "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
                "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
                "source": "The New York Times",
                "headline": {
                    "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
                    "kicker": null,
                    "content_kicker": null,
                    "print_headline": "The Attachment Diaries",
                    "name": null,
                    "seo": null,
                    "sub": null
                },
                "keywords": [
                    {
                        "name": "subject",
                        "value": "Movies",
                        "ra

In [6]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20) : 
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(nyt_api_query + "&page=" + str(page)).json()

    # It's nice to see some pretty print output.
    print()
    print("Page", page)
    print("===================================================================")
    
    # Try and save the reviews to the reviews_list
    while len(reviews["response"]["docs"]) > 0 : 

        # loop through the reviews["response"]["docs"] and append each review to the list
        review = reviews["response"]["docs"].pop()
        print("Headline: ", review["headline"]["main"])
        reviews_list.append(review)
        
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)




Page 0
Headline:  ‘Love in the Time of Fentanyl’ Review: Heartbreak, Death and Hope in Vancouver
Headline:  ‘Your Place or Mine’ Review: Try Neither
Headline:  ‘A Thousand and One’ Review: A New York Love Story
Headline:  ‘The Lost Weekend: A Love Story’ Review: When John Lennon Strayed
Headline:  ‘One True Loves’ Review: A Romance Lost at Sea
Headline:  ‘Other People’s Children’ Review: True Romance
Headline:  ‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip
Headline:  ‘You Can Live Forever’ Review: Do You Love Me Now?
Headline:  Review: ‘What’s Love Got to Do With It?’ Probably a Lot
Headline:  ‘The Attachment Diaries’ Review: Love, Sick

Page 1
Headline:  ‘My Policeman’ Review: Two Love Affairs, Equally Tragic
Headline:  ‘Bones and All’ Review: You Eat What You Are
Headline:  ‘The Inspection’ Review: Boot Camp, a Love Story
Headline:  ‘The Sound of Christmas’ Review: A Gospel Singer Finds Love
Headline:  ‘Lady Chatterley’s Lover’ Review: When Connie Met Ollie
Headline

In [7]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range(5) : print(json.dumps(reviews_list[i], indent=4))

{
    "web_url": "https://www.nytimes.com/2023/02/02/movies/love-in-the-time-of-fentanyl-review.html",
    "snippet": "To combat the overdose crisis, a group that includes former and current users open a safe consumption site where shooting up does not have to mean death.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018Love in the Time of Fentanyl\u2019 Review: Heartbreak, Death and Hope in Vancouver",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "Love in the Time Of Fentanyl",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Documentary Films and Programs",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "subject",
            "value": "Fentanyl",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "subject",
            "value": "Drug Abuse

In [8]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
nyt_df = pd.json_normalize(reviews_list)
nyt_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None
1,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
2,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/14/movies/fran...,François Ozon’s adaptation of Ernst Lubitsch’s...,The New York Times,"[{'name': 'creative_works', 'value': 'Frantz (...",2017-03-14T21:14:24+0000,708,"Review: ‘Frantz,’ a Mysterious Frenchman and t...",None,None,The Wounds of War and Love,None,None,None,By Stephen Holden,"[{'firstname': 'Stephen', 'middlename': None, ...",None
196,https://www.nytimes.com/2017/04/20/movies/the-...,"Oscar Isaac, Christian Bale and Charlotte Le B...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-04-20T16:49:46+0000,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
197,https://www.nytimes.com/2017/04/20/movies/the-...,"Inspired by a true story, “The Happiest Day in...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-04-20T20:45:54+0000,824,Review: In ‘The Happiest Day in the Life of Ol...,None,None,Love and Life on the Ropes,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
198,https://www.nytimes.com/2017/04/27/movies/haro...,Meet the husband-and-wife team that inspired s...,The New York Times,"[{'name': 'creative_works', 'value': 'Harold a...",2017-04-27T20:29:44+0000,263,Review: ‘Harold and Lillian’ Introduces a Holl...,None,None,Harold and Lillian: A Hollywood Love Story,None,None,None,By Monica Castillo,"[{'firstname': 'Monica', 'middlename': None, '...",None


In [9]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
nyt_df["title"] = nyt_df["headline.main"].str.extract(r"‘(.*)’")
nyt_df["title"] = nyt_df["title"].str.rstrip(",")
nyt_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None,Love in the Time of Fentanyl
1,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
2,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
4,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves


In [10]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
nyt_df["keywords"] = nyt_df["keywords"].apply(extract_keywords)
nyt_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,subject: Documentary Films and Programs;subjec...,2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None,Love in the Time of Fentanyl
1,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies;creative_works: Your Place or ...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
2,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
4,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves


In [11]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
movie_titles = nyt_df["title"].to_list()
movie_titles

['Love in the Time of Fentanyl',
 'Your Place or Mine',
 'A Thousand and One',
 'The Lost Weekend: A Love Story',
 'One True Loves',
 'Other People’s Children',
 'A Tourist’s Guide to Love',
 'You Can Live Forever',
 'What’s Love Got to Do With It?',
 'The Attachment Diaries',
 'My Policeman',
 'Bones and All',
 'The Inspection',
 'The Sound of Christmas',
 'Lady Chatterley’s Lover',
 'Nelly & Nadine',
 'Alcarràs',
 'After Love',
 'In From the Side',
 'Pamela, a Love Story',
 'Fire of Love',
 'Both Sides of the Blade',
 'Thor: Love and Thunder’ Review: A God',
 'The Wheel',
 'Art of Love',
 'Alone Together',
 'A Love Song',
 'I Love My Dad',
 'Waiting for Bojangles',
 'About Fate’ Review: Love the One You',
 'Love After Love',
 'La Mami',
 'See You Then',
 'I Love America',
 'Anaïs in Love',
 'In a New York Minute',
 'Dinner in America',
 'Benediction’ Review: A Poet',
 'Stay Prayed Up',
 'Love & Gelato',
 'Aulcie',
 'West Side Story',
 'A Journal for Jordan',
 'The Pink Cloud',
 'Ling

### Access The Movie Database API

In [12]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key


In [13]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
number_of_requests = 0

# Loop through the titles
for movie_title in movie_titles : 

    # Check if we need to sleep before making a request
    if number_of_requests >= 50 :
        number_of_requests = 0
        time.sleep(10) # As of December 2019, there is no longer a rate limit, according to the documentation.  But, whatever. :-)

    # Add 1 to the request counter
    else : 
        number_of_requests += 1
    
    # Perform a "GET" request for The Movie Database
    response_data = requests.get(url+str(movie_title)+tmdb_key_string).json()

    # Iterate through each of the potentially multiple movie ids.
    for movie in response_data["results"] : 

        genre_names = []
        spoken_languages = []
        production_countries = []

        # Make a request for a the full movie details
        full_details_url = "https://api.themoviedb.org/3/movie/" + str(movie["id"])
        full_details_key_string = "?api_key=" + tmdb_api_key

        # Execute "GET" request with url
        # Include a try clause to search for the full movie details.
        # Use the except clause to print out a statement if a movie
        # is not found.

        try:
            full_details = requests.get(full_details_url + full_details_key_string).json()
            #print(json.dumps(full_details, indent=4))

            # Print out the title that was found
            print("Movie ID " + str(movie["id"]) + " found.  \tTitle: " + movie["original_title"])

        except requests.exceptions.HTTPError: 
            print("Movie ID: " + str(movie["id"]) + " NOT FOUND!")
            break


        # Extract the genre names into a list
        for genre in full_details["genres"] : 
            genre_names.append(genre["name"])

        # Extract the spoken_languages' English name into a list
        for language in full_details["spoken_languages"] : 
            spoken_languages.append(language["english_name"])

        # Extract the production_countries' name into a list
        for country in full_details["production_countries"] : 
            production_countries.append(country["name"])

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
            
        # Note: The only way I could make the dict look like the workbook
        # example was to build this very specific dictionary below.
        # I'm not sure how practical it is to do something like this in 
        # a work situation.
        movie_dict = {
            "title"                 : full_details["title"],
            "original_title"        : full_details["original_title"],
            "budget"                : full_details["budget"],
            "genre"                 : genre_names,
            "language"              : full_details["original_language"],
            "spoken_languages"      : spoken_languages,
            "homepage"              : full_details["homepage"],
            "overview"              : full_details["overview"],
            "popularity"            : full_details["popularity"],
            "runtime"               : full_details["runtime"],
            "revenue"               : full_details["revenue"],
            "release_date"          : full_details["release_date"],
            "vote_average"          : full_details["vote_average"],
            "vote_count"            : full_details["vote_count"],
            "production_countries"  : production_countries
        }

        tmdb_movies_list.append(movie_dict)
        



Movie ID 976653 found.  	Title: Love in the Time of Fentanyl
Movie ID 703451 found.  	Title: Your Place or Mine
Movie ID 359490 found.  	Title: 每天愛您８小時
Movie ID 351800 found.  	Title: Your Place or Mine?
Movie ID 726421 found.  	Title: Your Place... or Mine?
Movie ID 855263 found.  	Title: A Thousand and One
Movie ID 85398 found.  	Title: 千夜一夜物語
Movie ID 46657 found.  	Title: Pohádky tisíce a jedné noci
Movie ID 90984 found.  	Title: A Thousand and One Nights
Movie ID 1211193 found.  	Title: 東宝千一夜
Movie ID 649798 found.  	Title: אלף נשותיו של נפתלי סימן טוב
Movie ID 963918 found.  	Title: وجد
Movie ID 327775 found.  	Title: ألف ليلة وليلة
Movie ID 1178472 found.  	Title: Mille e una Monica
Movie ID 290803 found.  	Title: Les mille et une mains
Movie ID 737187 found.  	Title: الف يد و يد
Movie ID 270840 found.  	Title: Isabel Allende. Erzähl mir vom Leben
Movie ID 437689 found.  	Title: Anteojito y Antifaz, mil intentos y un invento
Movie ID 964549 found.  	Title: Alfredo Arias, el homb

In [14]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list, indent=4))

[
    {
        "title": "Love in the Time of Fentanyl",
        "original_title": "Love in the Time of Fentanyl",
        "budget": 0,
        "genre": [
            "Documentary"
        ],
        "language": "en",
        "spoken_languages": [
            "English"
        ],
        "homepage": "",
        "overview": "An intimate portrait of a community fighting to save lives and keep hope alive in a neighborhood ravaged by the overdose crisis.",
        "popularity": 1.273,
        "runtime": 85,
        "revenue": 0,
        "release_date": "2023-02-03",
        "vote_average": 0.0,
        "vote_count": 0,
        "production_countries": [
            "Canada",
            "United States of America"
        ]
    },
    {
        "title": "Your Place or Mine",
        "original_title": "Your Place or Mine",
        "budget": 0,
        "genre": [
            "Romance",
            "Comedy"
        ],
        "language": "en",
        "spoken_languages": [
            "English"

In [15]:
# Convert the results to a DataFrame
tmdb_df = pd.json_normalize(tmdb_movies_list)
tmdb_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,[Documentary],en,[English],,An intimate portrait of a community fighting t...,1.273,85,0,2023-02-03,0.000,0,"[Canada, United States of America]"
1,Your Place or Mine,Your Place or Mine,0,"[Romance, Comedy]",en,[English],https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,23.476,109,0,2023-02-10,6.278,689,[United States of America]
2,Your Place or Mine!,每天愛您８小時,0,"[Comedy, Romance]",cn,"[Mandarin, Cantonese]",,Your Place or Mine! explores the difficulties ...,2.324,100,0,1998-09-25,4.800,4,[Hong Kong]
3,Your Place or Mine?,Your Place or Mine?,0,"[Romance, Comedy, Drama]",tl,"[English, Tagalog]",,One night with a total stranger. And fate brou...,0.685,89,0,2015-04-29,6.500,2,[Philippines]
4,Your Place... or Mine?,Your Place... or Mine?,0,"[Comedy, TV Movie]",en,[English],,Karen (Tyne Daly) is a San Francisco real esta...,1.460,100,0,1983-03-02,0.000,0,[United States of America]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1585,The Promise,The Promise,0,[],en,[],,Real friends keep their promises. Even if they...,0.600,8,0,2013-10-16,7.000,3,[]
1586,The Happiest Day in the Life of Olli Mäki,Hymyilevä mies,1330000,"[Drama, Romance]",fi,"[English, Finnish]",,Summer 1962 and Olli Mäki has a shot at the wo...,8.541,95,0,2016-09-02,6.700,81,"[Finland, Germany, Sweden]"
1587,Harold and Lillian: A Hollywood Love Story,Harold and Lillian: A Hollywood Love Story,0,[Documentary],en,[English],http://haroldandlillian.com,Working largely uncredited in the Hollywood sy...,3.924,96,0,2017-02-01,6.889,19,[United States of America]
1588,Hounds of Love,Hounds of Love,0,"[Crime, Drama, Horror]",en,[English],http://www.houndsoflovemovie.com/,When Vicki Maloney is randomly abducted from a...,10.133,108,0,2016-10-07,6.100,323,[Australia]


### Merge and Clean the Data for Export

In [26]:
# Merge the New York Times reviews and TMDB DataFrames on title
movies_df = pd.merge(tmdb_df, nyt_df, on="title")
movies_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,[Documentary],en,[English],,An intimate portrait of a community fighting t...,1.273,85,...,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None
1,Your Place or Mine,Your Place or Mine,0,"[Romance, Comedy]",en,[English],https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,23.476,109,...,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
2,A Thousand and One,A Thousand and One,0,"[Drama, Crime]",en,"[English, Portuguese, Spanish]",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.721,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[English],https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,1.942,95,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,"[English, Spanish]",,Emma and Jesse are living the perfect life tog...,14.569,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,The Promise,The Promise,0,[],en,[],,A man struggling with the loss of a relationsh...,0.600,0,...,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
512,The Promise,The Promise,0,[Documentary],en,[],,The Promise is an urban fairy tale that plays ...,0.600,6,...,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
513,The Promise,The Promise,0,[Documentary],ar,"[Arabic, English]",,A young Palestinian promises his father on his...,0.600,84,...,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
514,The Promise,The Promise,0,[],en,[],,Real friends keep their promises. Even if they...,0.600,8,...,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [28]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genre","spoken_languages"]

# Create a list of characters to remove
characters_to_remove = "[]"

# Loop through the list of columns to fix
for column in columns_to_fix : 
    # Convert the column to type 'str'
    movies_df[column] = movies_df[column].astype(str)

    # Loop through characters to remove
    movies_df[column] = movies_df[column].str.replace('[', '')
    movies_df[column] = movies_df[column].str.replace(']', '')

# Display the fixed DataFrame
movies_df.head(5)

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,'Documentary',en,'English',,An intimate portrait of a community fighting t...,1.273,85,...,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None
1,Your Place or Mine,Your Place or Mine,0,"'Romance', 'Comedy'",en,'English',https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,23.476,109,...,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
2,A Thousand and One,A Thousand and One,0,"'Drama', 'Crime'",en,"'English', 'Portuguese', 'Spanish'",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.721,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,'Documentary',en,'English',https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,1.942,95,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,One True Loves,One True Loves,0,"'Romance', 'Drama', 'Comedy'",en,"'English', 'Spanish'",,Emma and Jesse are living the perfect life tog...,14.569,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None


In [29]:
# Drop "byline.person" column
movies_df.drop(labels="byline.person", axis="columns")

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,'Documentary',en,'English',,An intimate portrait of a community fighting t...,1.273,85,...,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,None
1,Your Place or Mine,Your Place or Mine,0,"'Romance', 'Comedy'",en,'English',https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,23.476,109,...,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,None
2,A Thousand and One,A Thousand and One,0,"'Drama', 'Crime'",en,"'English', 'Portuguese', 'Spanish'",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.721,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,'Documentary',en,'English',https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,1.942,95,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,One True Loves,One True Loves,0,"'Romance', 'Drama', 'Comedy'",en,"'English', 'Spanish'",,Emma and Jesse are living the perfect life tog...,14.569,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,The Promise,The Promise,0,,en,,,A man struggling with the loss of a relationsh...,0.600,0,...,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,None
512,The Promise,The Promise,0,'Documentary',en,,,The Promise is an urban fairy tale that plays ...,0.600,6,...,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,None
513,The Promise,The Promise,0,'Documentary',ar,"'Arabic', 'English'",,A young Palestinian promises his father on his...,0.600,84,...,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,None
514,The Promise,The Promise,0,,en,,,Real friends keep their promises. Even if they...,0.600,8,...,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,None


In [38]:
# Delete duplicate rows and reset index
filtered_movies_df = movies_df.drop_duplicates(subset="title", keep="first")
filtered_movies_df.reset_index()

,index,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,0,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,'Documentary',en,'English',,An intimate portrait of a community fighting t...,1.273,...,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None
1,1,Your Place or Mine,Your Place or Mine,0,"'Romance', 'Comedy'",en,'English',https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,23.476,...,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
2,2,A Thousand and One,A Thousand and One,0,"'Drama', 'Crime'",en,"'English', 'Portuguese', 'Spanish'",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.721,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
3,3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,'Documentary',en,'English',https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,1.942,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,4,One True Loves,One True Loves,0,"'Romance', 'Drama', 'Comedy'",en,"'English', 'Spanish'",,Emma and Jesse are living the perfect life tog...,14.569,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,482,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"'Romance', 'Drama', 'War'",en,"'Turkish', 'English'",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",12.140,...,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
153,483,The Other Half,The Other Half,0,"'Drama', 'Romance'",en,'English',http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,3.016,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
154,496,Frantz,Frantz,0,"'History', 'Drama', 'Romance'",fr,"'German', 'French'",,"In the aftermath of WWI, a young German who gr...",14.406,...,"Review: ‘Frantz,’ a Mysterious Frenchman and t...",None,None,The Wounds of War and Love,None,None,None,By Stephen Holden,"[{'firstname': 'Stephen', 'middlename': None, ...",None
155,497,The Promise,The Promise,90000000,"'History', 'Romance', 'Drama'",en,"'Armenian', 'German', 'English', 'French'",http://www.survivalpictures.org/the-promise/,Set during the last days of the Ottoman Empire...,11.416,...,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [40]:
# Export data to CSV without the index
filtered_movies_df.to_csv("output.csv", index=False)